In [1]:
import pandas as pd

In [6]:
dealer_df = pd.read_csv('data/dealer_data.csv')

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (75,76,77,78,79,105,106,122,123,128,130,132,134,171,172,178,189) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
for c in dealer_df.columns:
    print(c)
    print(dealer_df[c][0])  
    print('---')

DefDate
2012-10-22
---
C2Dvolume_PAR
10000.0
---
C2D_Dealer
9ea411b1b98cf4084186ab303328c358f2189128
---
DealCSP
00126VAB6
---
Date
2012-10-24 10:21 AM
---
TradeSize
Retail
---
PrimaryDealer
0.0
---
TimeToMaturity
6.622162535
---
TimeSinceOffering
7.379901215
---
Offering_amount
5.164785974
---
Rating
17.0
---
JunkDummy
1.0
---
UnratedDummy
0.0
---
Enhanced
1.0
---
IssueID
476894.0
---
Redeemable
1.0
---
Callable
1.0
---
SinkingFund
0.0
---
Seniority
SeniorSecured
---
PostDummy
1.0
---
TotalVolume
12.22832814
---
PCA_cen
0.324942488
---
TimeCluster
201243.0
---
Year
2012.0
---
Month
10.0
---
YearMonth
201210.0
---
TopCentrality
0.0
---
0.840030434
100009ea411b1b98cf4084186ab303328c358f218912800126VAB641206.4313657407
---
FirmDef
Co_35941204
---
IssueIDDefaultevent
47689441204RiskRating
---
TradePreCount
97.0
---
TradePreVolume
33307000.0
---
INDUSTRY_CODE
10.0
---
Company
Co_359
---
trailingInventory_bond_30
0.0
---
trailingInventory_bond_alldealers_30
0.0
---
trailingInventory_dealer_

In [22]:
# List of relevant columns, including the recovery rate
relevant_columns = [
    'DealCSP', 'PrimaryDealer', 'C2D_Dealer', 'TradeSize', 'C2Ctrades', 'C2Dvolume_PAR', 
    'TradePreCount', 'TradePreVolume', 'TimeToMaturity', 'TimeSinceOffering', 'Default', 
    'TRADE_CHAIN2', 'TRADE_CHAIN3', 'TRADE_CHAIN4', 'TRADE_CHAIN5', 'TRADE_CHAIN6', 'TRADE_CHAIN7', 'TRADE_CHAIN8', 'TRADE_CHAIN9', 'TRADE_CHAIN10',
    'Markup', 'predicted_Primary', 'Rating', 'Commission', 'PostDummy', 'IndustryDefaultRate', 
    'UncertaintyIndex', 'PriceDistance', 'RR_Price'
]

# Limit the DataFrame to the relevant columns
dealer_df_limited = dealer_df[relevant_columns]

# rename DealCSP to CUSIP
dealer_df_limited.rename(columns={'DealCSP': 'CUSIP'}, inplace=True)

# Display the first few rows of the limited DataFrame
dealer_df_limited.head()

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,CUSIP,PrimaryDealer,C2D_Dealer,TradeSize,C2Ctrades,C2Dvolume_PAR,TradePreCount,TradePreVolume,TimeToMaturity,TimeSinceOffering,...,TRADE_CHAIN10,Markup,predicted_Primary,Rating,Commission,PostDummy,IndustryDefaultRate,UncertaintyIndex,PriceDistance,RR_Price
0,00126VAB6,0.0,9ea411b1b98cf4084186ab303328c358f2189128,Retail,4.0,10000.0,97.0,33307000.0,6.622163,7.379901,...,NaN,0.002085,0.151936,17.0,0.002085,1.0,0.832444,275.808527,-0.064324437,47.9600
1,00126VAB6,0.0,df8e2728f63ee1112e65ccc34a2e6dd006d9825e,SmallInstitutional,0.0,965000.0,98.0,33317000.0,6.620745,7.380565,...,NaN,-1.000000,0.007963,17.0,NaN,1.0,0.832444,275.808527,-0.024456522,46.0000
2,00126VAB6,0.0,9ea411b1b98cf4084186ab303328c358f2189128,Retail,4.0,10000.0,98.0,33317000.0,6.620559,7.380652,...,NaN,0.002085,0.151791,17.0,0.002085,1.0,0.832444,275.808527,-0.064324437,47.9600
3,00126VAB6,0.0,9ea411b1b98cf4084186ab303328c358f2189128,Retail,4.0,10000.0,99.0,34282000.0,6.619243,7.381267,...,NaN,0.004168,0.151671,17.0,0.004168,1.0,0.832444,275.808527,-0.062379206,47.8605
4,00126VAB6,0.0,4453917d43ee60954c145536e3ea07bc15324c59,LargeInstitutional,2.0,1000000.0,100.0,34292000.0,6.605873,7.387446,...,NaN,0.005464,0.076537,17.0,0.005464,1.0,0.832444,245.097169,-0.019125683,45.7500


In [20]:
credit_df = pd.read_csv('transcripts/credit_df.csv', delimiter='|')

In [23]:
credit_df.merge(dealer_df_limited, on=['CUSIP'], how='inner')

,RR,ActIndustryDistress1,ActIndustryDistress2,Senior secured,Senior unsecured,Senior subordinated,Subordinated \& Junior,Equity value,Default barrier,Net income margin,...,TRADE_CHAIN10,Markup,predicted_Primary,Rating_y,Commission,PostDummy,IndustryDefaultRate,UncertaintyIndex,PriceDistance,RR_Price
0,29.023972,0,1,0,1,0,0,635.1260,0.173560,-0.026785,...,NaN,0.038462,0.1560,13.0,0.038462,1.0,3.797222,188.355661,3.205128205,13.000
1,29.023972,0,1,0,1,0,0,635.1260,0.173560,-0.026785,...,NaN,0.031250,0.2140,13.0,0.000000,1.0,3.797222,188.355661,2.416666667,16.000
2,29.023972,0,1,0,1,0,0,635.1260,0.173560,-0.026785,...,NaN,0.034483,0.1560,13.0,0.034483,1.0,3.797222,188.355661,2.770114943,14.500
3,29.023972,0,1,0,1,0,0,635.1260,0.173560,-0.026785,...,NaN,-1.000000,0.3290,13.0,NaN,1.0,3.797222,188.355661,2.644444444,15.000
4,29.023972,0,1,0,1,0,0,635.1260,0.173560,-0.026785,...,NaN,0.034483,0.1560,13.0,0.034483,1.0,3.797222,188.355661,2.770114943,14.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85714,11.007645,1,1,0,1,0,0,35.2173,0.361788,-2.618239,...,NaN,-1.000000,0.1250,16.0,NaN,1.0,6.451000,82.562643,-0.195997856,11.194
85715,11.007645,1,1,0,1,0,0,35.2173,0.361788,-2.618239,...,NaN,-1.000000,0.1250,16.0,NaN,1.0,6.451000,82.562643,-0.195997856,11.194
85716,11.007645,1,1,0,1,0,0,35.2173,0.361788,-2.618239,...,NaN,-1.000000,0.1250,16.0,NaN,1.0,6.451000,82.562643,0.064081343,8.458
85717,11.007645,1,1,0,1,0,0,35.2173,0.361788,-2.618239,...,NaN,0.031250,0.0357,16.0,0.031250,1.0,6.451000,80.106604,0.125,8.000


In [25]:
credit_df.to_csv('transcripts/credit_dealer_df.csv', sep='|', index=False)